# Make Pyramid
Implement [ndpyramid](https://github.com/carbonplan/ndpyramid/tree/main) (Credit: Carbonplan) to create a pyramid datatree.datatree 'pyramid' in which each level has a higher resolution. Such pyramids are compatible with several Carbonplan webviewer and can be tested in [ncview](https://ncview-js.staging.carbonplan.org/)
## Imports 

In [1]:
import sys
import os.path
import numpy as np
import xarray as xr
import pandas as pd
from pathlib import Path

#### Import Resampling modules
Before running this code, make sure your S3 credentials are stored in the config.toml.

In [2]:
parent_dir = Path().resolve().parent
sys.path.append(str(parent_dir))
from resampling.config import Config
from resampling.load import write_zarr_s3
from resampling.transform import make_pyramid
from resampling.transform import expand_to_global_coverage
from resampling.extract import extract_private_s3_zarr

C:\Users\willem.boone\Documents\projects\Edito\venv2\Lib\site-packages\carbonplan_data\__init__.py:29: UserWarning: CARBONPLAN_DATA environment variable not set, `carbonplan.data.cat` may not work as expected.Known data locations include: ['https://storage.googleapis.com/carbonplan-data', 'https://carbonplan.blob.core.windows.net/carbonplan-data'].
  warnings.warn(msg)


## Open bathymetry ZARR

In [3]:
ds = extract_private_s3_zarr(name="DS20240807_bathymetry.zarr")
print(ds)

<xarray.Dataset> Size: 2MB
Dimensions:    (latitude: 400, longitude: 500)
Coordinates:
  * latitude   (latitude) float64 3kB 69.95 69.85 69.75 ... 30.25 30.15 30.05
  * longitude  (longitude) float64 4kB -9.95 -9.85 -9.75 ... 39.75 39.85 39.95
Data variables:
    elevation  (latitude, longitude) float64 2MB dask.array<chunksize=(200, 250), meta=np.ndarray>


#### Expand to global coverage, make sure the lat & lon step are conform the lat/lon resolution of the dataset

In [4]:
ds = expand_to_global_coverage(ds, step_lon=0.1, step_lat=0.1)
print(ds)

global_ds=<xarray.Dataset> Size: 52MB
Dimensions:    (latitude: 1801, longitude: 3601)
Coordinates:
  * latitude   (latitude) float64 14kB -90.0 -89.9 -89.8 ... 89.8 89.9 90.0
  * longitude  (longitude) float64 29kB -180.0 -179.9 -179.8 ... 179.9 180.0
Data variables:
    elevation  (latitude, longitude) float64 52MB nan nan nan ... nan nan nan
<xarray.Dataset> Size: 52MB
Dimensions:    (latitude: 1801, longitude: 3601)
Coordinates:
  * latitude   (latitude) float64 14kB -90.0 -89.9 -89.8 ... 89.8 89.9 90.0
  * longitude  (longitude) float64 29kB -180.0 -179.9 -179.8 ... 179.9 180.0
Data variables:
    elevation  (latitude, longitude) float64 52MB nan nan nan ... nan nan nan


#### Create the pyramid

In [5]:
merged_pyramid = make_pyramid(
    ds=ds,
    pixels_per_tile=128,
    version=0.17,
    levels=6,
)
print(merged_pyramid)

start reproject


TypeError: cannot directly convert an xarray.Dataset into a numpy array. Instead, create an xarray.DataArray first, either with indexing on the Dataset or by invoking the `to_dataarray()` method.